In [1]:
from google.colab import drive
drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Genai Project/augmented images/train/images/
%cd /content/drive/MyDrive/Genai Project/VLM
!pwd

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1euUzSZa1IjWGSY7kHFLr2B_d6lpqAzvE/Genai Project/VLM
/content/drive/.shortcut-targets-by-id/1euUzSZa1IjWGSY7kHFLr2B_d6lpqAzvE/Genai Project/VLM


In [ ]:
from PIL import Image
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
class VLMDataset(Dataset):

    def __init__(self, df, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.prompt_response_image = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.prompt_response_image)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.prompt_response_image['Image_name'][idx])
        image = Image.open(img_name)
        prompt = self.prompt_response_image['Prompt'][idx]
        response = self.prompt_response_image['Response'][idx]
        sample = {'image': image, 'prompt': prompt, 'response': response}

        if self.transform:
            sample = {'image': self.transform(sample['image']), 'prompt': prompt, 'response': response}

        return sample

In [ ]:
prompt_response_image = pd.read_csv('response_prompt_vlm_jpg.csv')
prompt_response_image_df =prompt_response_image.sample(frac=1,random_state = 1).reset_index(drop=True)
print(prompt_response_image_df.head())
vlm_dataset = VLMDataset(prompt_response_image_df, root_dir='images_jpg')

fig = plt.figure()

for i, sample in enumerate(vlm_dataset):
    print(i, sample['image'], sample['prompt'], sample['response'])
    # display(sample['image'])
    if i == 3:
        plt.show()
        break

   Sno                                             Prompt  \
0   15  Can you make a short description about the par...   
1   99  You are an expert in IoT and hardware with ext...   
2   76  You are an expert in IoT and hardware with ext...   
3   17  Can you make a short description about the par...   
4  132  You have been given an IoT kit component: 3-6V...   

                                            Response  \
0  Description:\nAn ultrasonic sensor is a device...   
1  Description:\r\nThe 104 pF ceramic capacitor i...   
2  Description:\r\nThe USA A to USB C adapter is ...   
3  Description:\r\nA Rotary Encoder Module is a s...   
4  Description:\r\nThe 3-6V motor is a compact, e...   

                     Image_name  
0         Ultrasonic_Sensor.jpg  
1  104_pF_ceramic_capacitor.jpg  
2            USB_A_to_USB_C.jpg  
3     Rotary_Encoder_Module.jpg  
4                3-6V_motor.jpg  
0 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4032x3024 at 0x7BEFFA677B80> Can yo

<Figure size 640x480 with 0 Axes>

In [ ]:
# from datasets import load_dataset
from PIL import Image
from io import BytesIO
import requests
import os
import json
import uuid




def process_and_save(dataset, output_folder, subset_name):
    # Define image subfolder within output folder
    subset_folder = os.path.join(output_folder, subset_name)
    image_subfolder = os.path.join(output_folder, 'images')


    if not os.path.exists(image_subfolder):
        os.makedirs(image_subfolder)


    if not os.path.exists(subset_folder):
        os.makedirs(subset_folder)


    # Initialize list to hold all JSON data
    json_data_list = []


    # Process and save images and labels
    for i,item in enumerate(dataset):
        if subset_name == 'val' and i<140:
          continue
        print('i',i)
        print('item: ',item)

        image = item['image']  # Assuming it's a PIL.Image object


        # Create a unique ID for each image
        unique_id = str(uuid.uuid4())


        # Define image path
        image_path = os.path.join(image_subfolder, f"{unique_id}.jpg")


        # Save image
        image.save(image_path)


        # Remove duplicates and format answers
        # answers = item['answers']
        answers = item['response']
        # unique_answers = list(set(answers))
        # formatted_answers = ", ".join(unique_answers)
        formatted_answers = answers


        # Structure for LLaVA JSON
        json_data = {
            "id": unique_id,
            "image": f"{unique_id}.jpg",
            "conversations": [
                {
                    "from": "human",
                    "value": "<image>\n" + item['prompt']
                },
                {
                    "from": "gpt",
                    "value": formatted_answers
                }
            ]
        }


        # Append to list
        json_data_list.append(json_data)
        if subset_name == 'train':
          if i==139:
            break
        elif subset_name == 'val':
          if i==149:
            break


    # Save the JSON data list to a file
    json_output_path = os.path.join(output_folder, subset_name, 'dataset.json')
    with open(json_output_path, 'w') as json_file:
        json.dump(json_data_list, json_file, indent=4)


def save_dataset(dataset_name, output_folder, subset_name):
    # Load the dataset from Hugging Face
    # dataset = load_dataset(dataset_name, split=subset_name)
    dataset = dataset_name
    # Process and save the datasets
    for subset_name,data in [(subset_name,dataset)]:
        if data:
            process_and_save(data, output_folder, subset_name)




# Usage example
output_folder = 'dataset'
save_dataset(vlm_dataset, output_folder, 'train')
save_dataset(vlm_dataset, output_folder, 'val')


i 0
item:  {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4032x3024 at 0x7BEFFA64BEB0>, 'prompt': 'Can you make a short description about the part: Ultrasonic Sensor\n\nIt should be divided into 3 sections as follows:\n1) Description\n2) Applications\n3) technical specification', 'response': 'Description:\nAn ultrasonic sensor is a device that measures the distance to an object by emitting ultrasonic sound waves and then detecting the waves that bounce back after hitting the object. It works on the principle of echolocation, similar to how bats navigate. The sensor typically consists of a transmitter, which emits ultrasonic waves, and a receiver, which detects the reflected waves. By measuring the time it takes for the waves to return, the sensor can calculate the distance to the object with high accuracy.\nApplications:\nUltrasonic sensors find applications in various fields, including robotics, automotive industry, healthcare, and industrial automation. In robotics,

In [2]:
%cd /content/drive/MyDrive/Genai Project/VLM/LLAVA_zip/LLaVA-main/
!pwd
!pip install -e .

/content/drive/.shortcut-targets-by-id/1euUzSZa1IjWGSY7kHFLr2B_d6lpqAzvE/Genai Project/VLM/LLAVA_zip/LLaVA-main
/content/drive/.shortcut-targets-by-id/1euUzSZa1IjWGSY7kHFLr2B_d6lpqAzvE/Genai Project/VLM/LLAVA_zip/LLaVA-main
Obtaining file:///content/drive/.shortcut-targets-by-id/1euUzSZa1IjWGSY7kHFLr2B_d6lpqAzvE/Genai%20Project/VLM/LLAVA_zip/LLaVA-main
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 34.1 MB/s et

In [3]:
!pip install -e ".[train]"
!pip install flash-attn --no-build-isolation

Obtaining file:///content/drive/.shortcut-targets-by-id/1euUzSZa1IjWGSY7kHFLr2B_d6lpqAzvE/Genai%20Project/VLM/LLAVA_zip/LLaVA-main
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━

In [ ]:
# !deepspeed llava/train/train_mem.py \
#     --lora_enable True --lora_r 128 --lora_alpha 256 --mm_projector_lr 2e-5 \
#     --deepspeed ./scripts/zero3.json \
#     --model_name_or_path lmsys/vicuna-13b-v1.5 \
#     --version v1 \
#     --data_path ./playground/data/llava_v1_5_mix665k.json \
#     --image_folder ./playground/data \
#     --vision_tower openai/clip-vit-large-patch14-336 \
#     --pretrain_mm_mlp_adapter ./checkpoints/llava-v1.5-13b-pretrain/mm_projector.bin \
#     --mm_projector_type mlp2x_gelu \
#     --mm_vision_select_layer -2 \
#     --mm_use_im_start_end False \
#     --mm_use_im_patch_token False \
#     --image_aspect_ratio pad \
#     --group_by_modality_length True \
#     --bf16 False \
#     --output_dir ./checkpoints/llava-v1.5-13b-lora \
#     --num_train_epochs 1 \
#     --per_device_train_batch_size 16 \
#     --per_device_eval_batch_size 4 \
#     --gradient_accumulation_steps 1 \
#     --evaluation_strategy "no" \
#     --save_strategy "steps" \
#     --save_steps 50000 \
#     --save_total_limit 1 \
#     --learning_rate 2e-4 \
#     --weight_decay 0. \
#     --warmup_ratio 0.03 \
#     --lr_scheduler_type "cosine" \
#     --logging_steps 1 \
#     --tf32 False \
#     --model_max_length 2048 \
#     --gradient_checkpointing True \
#     --dataloader_num_workers 4 \
#     --lazy_preprocess True \
#     --report_to wandb

#Set true if using A100 GPU: bf16 and tf32

[2024-04-28 20:47:05,722] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2024-04-28 20:47:08.305456: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 20:47:08.305504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 20:47:08.306779: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 20:47:09.479440: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2024-04-28 20:47:10,466] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training wi

In [ ]:
# !deepspeed llava/train/train_mem.py \
#     --deepspeed scripts/zero2.json \
#     --lora_enable True \
#     --lora_r 128 \
#     --lora_alpha 256 \
#     --mm_projector_lr 2e-5 \
#     -- bits 4 \
#     --model_name_or_path liuhaotian/llava-v1.5-7b \
#     --version llava_llama_2 \
#     --data_path ../../dataset/train/dataset.json \
#     --image_folder ../../dataset/images/ \
#     --vision_tower openai/clip-vit-large-patch14-336 \
#     --mm_projector_type mlp2x_gelu \
#     --mm_vision_select_layer -2 \
#     --mm_use_im_start_end False \
#     --mm_use_im_patch_token False \
#     --image_aspect_ratio pad \
#     --group_by_modality_length True \
#     --bf16 True \
#     --output_dir llava/checkpoints/llava-v1.5-7b-lora \
#     --num_train_epochs 2 \
#     --per_device_train_batch_size 8 \
#     --per_device_eval_batch_size 8 \
#     --gradient_accumulation_steps 16 \
#     --evaluation_strategy 'no' \
#     --save_strategy "steps" \
#     --save_steps 50000 \
#     --save_total_limit 1 \
#     --learning_rate 2e-4 \
#     --weight_decay 0. \
#     --warmup_ratio 0.03 \
#     --lr_scheduler_type "cosine" \
#     --logging_steps 1 \
#     --tf32 True \
#     --model_max_length 2048 \
#     --gradient_checkpointing True \
#     --dataloader_num_workers 4 \
#     --lazy_preprocess True \
#     --report_to wandb

In [4]:
!deepspeed llava/train/train_mem.py \
    --deepspeed scripts/zero2.json \
    --lora_enable True \
    --lora_r 64 \
    --lora_alpha 128 \
    --mm_projector_lr 2e-5 \
    --model_name_or_path liuhaotian/llava-v1.5-7b \
    --version llava_llama_2 \
    --data_path ../../dataset/train/dataset.json \
    --image_folder ../../dataset/images/ \
    --vision_tower openai/clip-vit-large-patch14-336 \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir llava/checkpoints/llava-v1.5-7b-lora \
    --num_train_epochs 10 \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 32 \
    --evaluation_strategy 'no' \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-4 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb


[2024-04-30 21:12:52,621] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2024-04-30 21:12:56.099896: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 21:12:56.099952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 21:12:56.101789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-30 21:12:57.320925: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2024-04-30 21:12:58,351] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training wi

In [2]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

# model_path = "liuhaotian/llava-v1.5-7b"

# tokenizer, model, image_processor, context_len = load_pretrained_model(
#     model_path=model_path,
#     model_base=None,
#     model_name=get_model_name_from_path(model_path)
# )


[2024-04-30 23:04:49,975] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [1]:
%cd /content/drive/MyDrive/Genai Project/VLM/LLAVA_zip/LLaVA-main/

/content/drive/.shortcut-targets-by-id/1euUzSZa1IjWGSY7kHFLr2B_d6lpqAzvE/Genai Project/VLM/LLAVA_zip/LLaVA-main


In [4]:
### VLM No object detector Zero shot HiLetgo GY-521 module

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"
# prompt = "Can you make a short description about the part: HiLetgo GY-521 module.\n It should be divided into 3 sections as follows: \n1) Description \n2) Applications \n3) technical specification"
image_file = "../../images_jpg/HiLetgo_GY-521_module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/HiLetgo_GY-521_module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7AC78D023B80>]
image sizes:  [(4032, 3024)]
Description:
The component is a small electronic module with a microchip, which is a crucial component in IoT devices. It is designed to be a versatile and compact solution for various IoT applications. The microchip is responsible for processing data, controlling the device, and communicating with other devices or systems.

Applications:
This component can be used in a wide range of IoT applications, such as smart home automation, industrial control systems, wearable devices, and smart city infrastructure. Its versatility and compact size make it suitable for various IoT projects, from simple home automation systems to complex industrial control systems.

Technical specifications:

1. Microchip: The microchip used in this component is a powerful and efficient processor that can handle various tasks, such as data processi

In [5]:
### VLM Zero shot HiLetgo GY-521 module

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: HiLetgo GY-521 module. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"
# prompt = "Can you make a short description about the part: HiLetgo GY-521 module.\n It should be divided into 3 sections as follows: \n1) Description \n2) Applications \n3) technical specification"
image_file = "../../images_jpg/HiLetgo_GY-521_module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/HiLetgo_GY-521_module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7FBE53BD3C40>]
image sizes:  [(4032, 3024)]
1. Description:
The HiLetgo GY-521 module is an IoT development board that is designed for use with the Arduino platform. It is a compact and versatile device that can be used for various IoT projects, including temperature and humidity monitoring, home automation, and remote control. The module features a built-in Wi-Fi module, which allows it to connect to the internet and communicate with other devices. Additionally, it has a GY-521 module that supports GSM/GPRS communication, which enables it to send and receive SMS messages.

2. Applications:
The HiLetgo GY-521 module can be used for a wide range of IoT applications, including:

* Smart home automation: The module can be used to control various devices in a home, such as lights, appliances, and security systems.
* Environmental monitoring: The module can be used to m

In [3]:
### VLM One shot HiLetgo GY-521 module

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: HiLetgo GY-521 module. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications.\n Here is an example for reference,\nULN 2003 Stepper Motor Driver Module:\nDescription:\nThe ULN2003 Stepper Motor Driver Module is an integrated circuit module designed to control bipolar stepper motors conveniently. It utilizes the ULN2003A Darlington array as its core component. This module provides an easy interface for controlling stepper motors, offering both simplicity and efficiency in motor control applications.\nApplications:\nThe ULN2003 Stepper Motor Driver Module finds widespread use in various projects and applications, including:\nRobotics: for precise control of robot movements.\nCNC machines: for controlling the motion of axes.\n3D printers: for controlling the position of the printing head.\nAutomated systems: for controlling mechanisms like conveyor belts and automated doors.\nDIY projects: for hobbyists and enthusiasts working on motion control projects.\nTechnical Specifications:\nOperating Voltage: 5V DC\nOutput Current: up to 500mA per channel (with proper heat sinking)\nDriver IC: ULN2003A Darlington Array\nNumber of Channels: 7 (suitable for driving bipolar stepper motors)\nCompatible Motors: Bipolar stepper motors with 4 or 6 wires\nControl Interface: 4-phase, 5-wire interface (can be easily interfaced with microcontrollers like Arduino)\nDimensions: Compact module size for easy integration into projects\nProtection Features: Built-in transient suppression diodes for protecting the module and connected components against voltage spikes\nMounting Holes: Provided for easy installation and integration into projects"
# prompt = "Can you make a short description about the part: HiLetgo GY-521 module.\n It should be divided into 3 sections as follows: \n1) Description \n2) Applications \n3) technical specification"
image_file = "../../images_jpg/HiLetgo_GY-521_module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/HiLetgo_GY-521_module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x799543D5FD30>]
image sizes:  [(4032, 3024)]
Description:
The HiLetgo GY-521 module is an integrated circuit module designed for use in IoT projects. It is a low-cost, versatile module that can be used for various applications, including wireless communication, data transmission, and sensor interfacing.

Applications:
The GY-521 module can be used in a wide range of IoT projects, such as home automation, industrial control systems, and smart city applications. It can be used to transmit sensor data, control actuators, and communicate with other devices wirelessly.

Technical specifications:

* Operating Voltage: 3.3V to 5.5V DC
* Data Rate: 250kbps
* Transmission Range: 100m (line of sight)
* Frequency Band: 2.4GHz
* Channels: 1
* Data Format: Modulated RF
* Power Source: External power supply (5V DC)
* Dimensions: 50mm x 30mm
* Weight: 20g
* Package: SOIC-8
* Compa

In [5]:
### VLM finetuning HiLetgo GY-521 module

# model_path = "llava/checkpoints/llava-v1.5-7b-task-lora-first"
model_path = "llava/checkpoints/llava-v1.5-7b-lora"
model_basee = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: HiLetgo GY-521 module. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"
image_file = "../../images_jpg/HiLetgo_GY-521_module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": model_basee,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512
})()

eval_model(args)

model_name:  llava-v1.5-7b-lora
Loading LLaVA from base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...
image files:  ['../../images_jpg/HiLetgo_GY-521_module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7A044948C910>]
image sizes:  [(4032, 3024)]
Description:
The HiLetgo GY-521 module is a compact and versatile sensor module designed for use in Internet of Things (IoT) projects. It features a built-in accelerometer and magnetometer, which allows it to detect motion and orientation. The module also includes a temperature sensor, making it suitable for monitoring environmental conditions. Additionally, it has a digital interface for easy communication with microcontrollers or other devices.
Applications:
The GY-521 module finds applications in various fields, including robotics, unmanned aerial vehicles (UAVs), automotive, gaming, and wearable technology. It can be used to detect motion and orientation in robotic platforms, stabilize UAVs during flight, monitor veh

In [1]:
%cd /content/drive/MyDrive/Genai Project/VLM/LLAVA_zip/LLaVA-main/

from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

/content/drive/.shortcut-targets-by-id/1euUzSZa1IjWGSY7kHFLr2B_d6lpqAzvE/Genai Project/VLM/LLAVA_zip/LLaVA-main
[2024-04-30 23:17:49,405] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
### VLM No object detector Zero shot RC522 RFID Module

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"
image_file = "../../images_jpg/RC522_RFID_Module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/RC522_RFID_Module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7D8460017E80>]
image sizes:  [(4032, 3024)]
The IoT kit component is a small blue device with a white sticker on it, which appears to be a Wi-Fi module. It is designed to connect to Wi-Fi networks and enable wireless communication between devices. The device has a USB interface, which allows it to be connected to a computer or other devices for data transfer and power supply.

1. Description: The Wi-Fi module is a small electronic component that enables wireless communication between devices. It is designed to connect to Wi-Fi networks and can be used to transmit data over the internet. The device has a USB interface, which allows it to be connected to a computer or other devices for data transfer and power supply.
2. Applications: The Wi-Fi module can be used in various IoT applications, such as home automation, industrial control systems, and smart cities. It can be 

In [2]:
### VLM Zero shot RC522 RFID Module

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: RC522 RFID Module. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"

image_file = "../../images_jpg/RC522_RFID_Module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/RC522_RFID_Module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x796DE30EBE50>]
image sizes:  [(4032, 3024)]
1. Description:
The RC522 RFID module is an RFID reader designed for use in IoT applications. It is a small, compact device that can be easily integrated into various projects. The module is equipped with an antenna and a microcontroller, which enables it to read RFID tags from a distance of up to 10 cm. The RC522 module can be used to read and write data on RFID tags, making it a versatile component for various IoT applications.

2. Applications:
The RC522 RFID module can be used in a wide range of applications, including access control, inventory management, asset tracking, and identification systems. It can be integrated into security systems, such as gates and doors, to allow authorized personnel to access restricted areas. In inventory management, the module can be used to track the movement of goods and materials in war

In [2]:
### VLM One shot RC522 RFID Module

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: RC522 RFID Module. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications.\n Here is an example for reference,\nULN 2003 Stepper Motor Driver Module:\nDescription:\nThe ULN2003 Stepper Motor Driver Module is an integrated circuit module designed to control bipolar stepper motors conveniently. It utilizes the ULN2003A Darlington array as its core component. This module provides an easy interface for controlling stepper motors, offering both simplicity and efficiency in motor control applications.\nApplications:\nThe ULN2003 Stepper Motor Driver Module finds widespread use in various projects and applications, including:\nRobotics: for precise control of robot movements.\nCNC machines: for controlling the motion of axes.\n3D printers: for controlling the position of the printing head.\nAutomated systems: for controlling mechanisms like conveyor belts and automated doors.\nDIY projects: for hobbyists and enthusiasts working on motion control projects.\nTechnical Specifications:\nOperating Voltage: 5V DC\nOutput Current: up to 500mA per channel (with proper heat sinking)\nDriver IC: ULN2003A Darlington Array\nNumber of Channels: 7 (suitable for driving bipolar stepper motors)\nCompatible Motors: Bipolar stepper motors with 4 or 6 wires\nControl Interface: 4-phase, 5-wire interface (can be easily interfaced with microcontrollers like Arduino)\nDimensions: Compact module size for easy integration into projects\nProtection Features: Built-in transient suppression diodes for protecting the module and connected components against voltage spikes\nMounting Holes: Provided for easy installation and integration into projects"
# prompt = "Can you make a short description about the part: HiLetgo GY-521 module.\n It should be divided into 3 sections as follows: \n1) Description \n2) Applications \n3) technical specification"
image_file = "../../images_jpg/RC522_RFID_Module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/RC522_RFID_Module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7C19BD505660>]
image sizes:  [(4032, 3024)]
Description:
The RC522 RFID module is an integrated circuit designed for reading and writing RFID tags. It is a compact and versatile module that can be easily integrated into various IoT projects. The RC522 module is compatible with both 125kHz and 13.56MHz RFID tags, making it suitable for a wide range of applications.

Applications:
The RC522 RFID module can be used in a variety of applications, including access control, inventory management, asset tracking, and payment systems. It can be integrated into security systems, such as gates and doors, to control access to restricted areas. In inventory management, it can be used to track the movement of goods and products in warehouses and retail stores. Additionally, it can be employed in asset tracking systems to monitor the movement and status of vehicles, equipment, and oth

In [3]:
### VLM finetuning RC522 RFID Module

# model_path = "llava/checkpoints/llava-v1.5-7b-task-lora-first"
model_path = "llava/checkpoints/llava-v1.5-7b-lora"
model_basee = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: RC522 RFID Module. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"
image_file = "../../images_jpg/RC522_RFID_Module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": model_basee,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512
})()

eval_model(args)

model_name:  llava-v1.5-7b-lora


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading LLaVA from base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...
image files:  ['../../images_jpg/RC522_RFID_Module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7C7C9937D7E0>]
image sizes:  [(4032, 3024)]
Description:
The RC522 RFID (Radio Frequency Identification) Module is a compact and versatile electronic component designed for use in IoT (Internet of Things) projects. It utilizes RFID technology to identify and authenticate objects or individuals based on their unique RFID tags. The module typically consists of a reader antenna, a microcontroller, and a set of interfaces for communication and control.
Applications:
The RC522 RFID Module finds applications in various industries and sectors, including:
Access control: Secure entry to buildings, offices, and restricted areas using RFID-enabled cards or fobs.
Asset tracking: Track and manage assets such as vehicles, equipment, and inventory using RFID tags.
Payment systems: 

In [1]:
%cd /content/drive/MyDrive/Genai Project/VLM/LLAVA_zip/LLaVA-main/

from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

/content/drive/.shortcut-targets-by-id/1euUzSZa1IjWGSY7kHFLr2B_d6lpqAzvE/Genai Project/VLM/LLAVA_zip/LLaVA-main
[2024-04-30 23:29:25,855] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
### VLM No object detector Zero shot Servo Motor SG90

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"
image_file = "../../images_jpg/Servo_Motor_SG90.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/Servo_Motor_SG90.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x78B190D77F70>]
image sizes:  [(4032, 3024)]
Description:
The component is a small electronic device, possibly a sensor or a microcontroller, with a blue casing. It is designed for use in IoT applications and is placed on a white surface. The device appears to be a small part of a larger IoT kit.

Applications:
This component can be used in various IoT applications, such as home automation, industrial monitoring, environmental monitoring, and more. Its small size and versatile design make it suitable for integration into different types of devices and systems.

Technical specifications:
The device is a small electronic component with a blue casing, and it is designed for use in IoT applications. It is likely to have a microcontroller or sensor functionality, and it may have a range of input/output options, such as digital or analog ports, and communication protocols like

In [2]:
### VLM Zero shot Servo Motor SG90

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: Servo Motor SG90. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"

image_file = "../../images_jpg/Servo_Motor_SG90.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/Servo_Motor_SG90.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x792CAEAEBF10>]
image sizes:  [(4032, 3024)]
1. Description:
The Servo Motor SG90 is a small, compact, and versatile motor that is commonly used in IoT devices and robotics projects. It is a brushless DC motor with a built-in gearbox, which allows for precise control of the motor's rotation speed and direction. The SG90 is designed to be easy to assemble and disassemble, making it a popular choice for DIY projects and educational purposes.

2. Applications:
The Servo Motor SG90 can be used in a wide range of applications, including robotics, automation, and IoT devices. Some common uses for this motor include:
- Controlling the movement of robots or drones
- Automating machines or equipment in factories or warehouses
- Controlling the position of cameras or other sensors in IoT devices
- Moving parts in 3D printers or CNC machines

3. Technical specifications:
The Servo 

In [2]:
### VLM One shot Servo Motor SG90

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: Servo Motor SG90. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications.\n Here is an example for reference,\nULN 2003 Stepper Motor Driver Module:\nDescription:\nThe ULN2003 Stepper Motor Driver Module is an integrated circuit module designed to control bipolar stepper motors conveniently. It utilizes the ULN2003A Darlington array as its core component. This module provides an easy interface for controlling stepper motors, offering both simplicity and efficiency in motor control applications.\nApplications:\nThe ULN2003 Stepper Motor Driver Module finds widespread use in various projects and applications, including:\nRobotics: for precise control of robot movements.\nCNC machines: for controlling the motion of axes.\n3D printers: for controlling the position of the printing head.\nAutomated systems: for controlling mechanisms like conveyor belts and automated doors.\nDIY projects: for hobbyists and enthusiasts working on motion control projects.\nTechnical Specifications:\nOperating Voltage: 5V DC\nOutput Current: up to 500mA per channel (with proper heat sinking)\nDriver IC: ULN2003A Darlington Array\nNumber of Channels: 7 (suitable for driving bipolar stepper motors)\nCompatible Motors: Bipolar stepper motors with 4 or 6 wires\nControl Interface: 4-phase, 5-wire interface (can be easily interfaced with microcontrollers like Arduino)\nDimensions: Compact module size for easy integration into projects\nProtection Features: Built-in transient suppression diodes for protecting the module and connected components against voltage spikes\nMounting Holes: Provided for easy installation and integration into projects"
# prompt = "Can you make a short description about the part: HiLetgo GY-521 module.\n It should be divided into 3 sections as follows: \n1) Description \n2) Applications \n3) technical specification"
image_file = "../../images_jpg/Servo_Motor_SG90.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/Servo_Motor_SG90.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7B49B3617A60>]
image sizes:  [(4032, 3024)]
Description:
The Servo Motor SG90 is a compact and versatile motor driver module designed for use with servo motors. It is a simple and cost-effective solution for controlling the rotation of servo motors in various applications.

Applications:
The Servo Motor SG90 is widely used in a variety of applications, including robotics, CNC machines, 3D printers, automated systems, and DIY projects. Its compact size and ease of use make it a popular choice for controlling the movement of mechanisms that require precise and smooth control.

Technical Specifications:

1. Operating Voltage: 5V DC
2. Continuous Current: 1.5A
3. Peak Current: 2.5A
4. Driver IC: SG90
5. Number of Channels: 1
6. Compatible Motors: Servo motors with a torque of up to 10 kg-cm
7. Control Interface: PWM signal (can be easily interfaced with microcontrollers lik

In [2]:
### VLM finetuning Servo Motor SG90

# model_path = "llava/checkpoints/llava-v1.5-7b-task-lora-first"
model_path = "llava/checkpoints/llava-v1.5-7b-lora"
model_basee = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: Servo Motor SG90. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"
image_file = "../../images_jpg/Servo_Motor_SG90.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": model_basee,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512
})()

eval_model(args)

model_name:  llava-v1.5-7b-lora


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading LLaVA from base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...
image files:  ['../../images_jpg/Servo_Motor_SG90.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x79025A85FD00>]
image sizes:  [(4032, 3024)]
Description:
The Servo Motor SG90 is a compact and versatile motor commonly used in robotics, automation, and IoT projects. It features a small form factor, making it ideal for space-constrained applications. The motor is equipped with a potentiometer for precise positioning and control. It typically consists of a motor, gearbox, potentiometer, and mounting brackets, all packed into a compact housing.
Applications:
Robotics: Servo motors are widely used in robotics for controlling movement and positioning of robotic arms, grippers, and other mechanical components.
Automation: They find applications in automated systems for controlling valves, dampers, and other mechanical devices.
IoT: Servo motors are employed in IoT project

In [1]:
%cd /content/drive/MyDrive/Genai Project/VLM/LLAVA_zip/LLaVA-main/

from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

/content/drive/.shortcut-targets-by-id/1euUzSZa1IjWGSY7kHFLr2B_d6lpqAzvE/Genai Project/VLM/LLAVA_zip/LLaVA-main
[2024-04-30 23:35:59,420] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
### VLM No object detector Zero shot Stepper Motor

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"
image_file = "../../images_jpg/Stepper_Motor.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/Stepper_Motor.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7E14535CB760>]
image sizes:  [(4032, 3024)]
Description:
The component is a small electronic device with a blue housing and an orange wire. It appears to be a power supply or a control module for an IoT device. The device is attached to a white wall, and it has a yellow light on the bottom.

Applications:
This component can be used in various IoT applications, such as home automation, industrial control systems, or smart city infrastructure. It can be used to power and control other devices, such as sensors, actuators, or communication modules, to enable remote monitoring and control of different systems.

Technical specifications:
The device has a blue housing, which is likely to be a protective casing for the electronic components inside. The orange wire is likely to be a power supply or a control signal wire, which can be connected to other devices or sensors. The yello

In [3]:
### VLM Zero shot Stepper Motor

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: Stepper Motor. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"

image_file = "../../images_jpg/Stepper_Motor.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/Stepper_Motor.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7CA3A8E2FD30>]
image sizes:  [(4032, 3024)]
1. Description:
A stepper motor is a type of electric motor that moves in discrete steps, rather than continuously like a regular motor. It is commonly used in IoT devices and hardware for precise positioning and control. Stepper motors are often found in applications such as robotics, CNC machines, and automation systems.

2. Applications:
Stepper motors are widely used in various IoT and hardware applications due to their precise and accurate movement. Some common applications include:
- Robotics: Stepper motors are used in robots for precise positioning and movement control.
- CNC machines: They are used in CNC machines for accurate positioning of cutting tools and other components.
- Automation systems: Stepper motors are used in automation systems for precise control of mechanical components.
- 3D printers: They are used in 

In [2]:
### VLM One shot Stepper Motor

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: Stepper Motor. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications.\n Here is an example for reference,\nULN 2003 Stepper Motor Driver Module:\nDescription:\nThe ULN2003 Stepper Motor Driver Module is an integrated circuit module designed to control bipolar stepper motors conveniently. It utilizes the ULN2003A Darlington array as its core component. This module provides an easy interface for controlling stepper motors, offering both simplicity and efficiency in motor control applications.\nApplications:\nThe ULN2003 Stepper Motor Driver Module finds widespread use in various projects and applications, including:\nRobotics: for precise control of robot movements.\nCNC machines: for controlling the motion of axes.\n3D printers: for controlling the position of the printing head.\nAutomated systems: for controlling mechanisms like conveyor belts and automated doors.\nDIY projects: for hobbyists and enthusiasts working on motion control projects.\nTechnical Specifications:\nOperating Voltage: 5V DC\nOutput Current: up to 500mA per channel (with proper heat sinking)\nDriver IC: ULN2003A Darlington Array\nNumber of Channels: 7 (suitable for driving bipolar stepper motors)\nCompatible Motors: Bipolar stepper motors with 4 or 6 wires\nControl Interface: 4-phase, 5-wire interface (can be easily interfaced with microcontrollers like Arduino)\nDimensions: Compact module size for easy integration into projects\nProtection Features: Built-in transient suppression diodes for protecting the module and connected components against voltage spikes\nMounting Holes: Provided for easy installation and integration into projects"
# prompt = "Can you make a short description about the part: HiLetgo GY-521 module.\n It should be divided into 3 sections as follows: \n1) Description \n2) Applications \n3) technical specification"
image_file = "../../images_jpg/Stepper_Motor.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/Stepper_Motor.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7F535143DC00>]
image sizes:  [(4032, 3024)]
Description:
The stepper motor is a type of electric motor that is designed to rotate in small, precise steps. It is commonly used in various applications, including robotics, CNC machines, 3D printers, and automated systems. The stepper motor operates by moving a rotor with a fixed number of steps (usually 200 or 400) in response to a series of electrical pulses.

Applications:
Stepper motors are widely used in various applications due to their precise and accurate control, making them ideal for projects that require precise movement and positioning. Some common applications include:

1. Robotics: Stepper motors are commonly used in robotics to control the movement of robotic arms, legs, and other components.
2. CNC machines: Stepper motors are used to control the movement of machine tools, such as milling machines and lathes.
3

In [2]:
### VLM finetuning Stepper Motor

# model_path = "llava/checkpoints/llava-v1.5-7b-task-lora-first"
model_path = "llava/checkpoints/llava-v1.5-7b-lora"
model_basee = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: Stepper Motor. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"
image_file = "../../images_jpg/Stepper_Motor.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": model_basee,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512
})()

eval_model(args)

model_name:  llava-v1.5-7b-lora


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading LLaVA from base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...
image files:  ['../../images_jpg/Stepper_Motor.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7A454168EB90>]
image sizes:  [(4032, 3024)]
Description:
A stepper motor is a type of brushless DC electric motor that divides a full rotation into a number of equal steps. It is commonly used in IoT (Internet of Things) applications due to its precise control and ability to move in discrete increments. Stepper motors consist of a rotor and a stator, with the rotor typically having a series of permanent magnets arranged around its circumference. The stator contains a set of coils that are energized in a specific sequence to produce the rotational force.
Applications:
Stepper motors find wide applications in various IoT and hardware projects due to their precise control and ability to move in discrete increments. Some common applications include:
Robotics: Stepper motors a

In [1]:
%cd /content/drive/MyDrive/Genai Project/VLM/LLAVA_zip/LLaVA-main/

from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

/content/drive/.shortcut-targets-by-id/1euUzSZa1IjWGSY7kHFLr2B_d6lpqAzvE/Genai Project/VLM/LLAVA_zip/LLaVA-main
[2024-04-30 23:41:38,183] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
### VLM No object detector Zero shot Water Level Detection Sensor Module

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"
image_file = "../../images_jpg/Water_Level_Detection_Sensor_Module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/Water_Level_Detection_Sensor_Module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7FDF9645B880>]
image sizes:  [(4032, 3024)]
Description:
The component is a small electronic device with a red casing, likely a microchip or a module. It has a few pins on the bottom, which are used for connecting to other components or devices. The device appears to be a part of an IoT kit, as it is designed to be used in IoT applications.

Applications:
This component can be used in various IoT applications, such as monitoring and controlling devices, data collection, and communication between different devices. It can be integrated into smart home systems, industrial automation, or any other IoT-based project that requires interconnectivity and data processing.

Technical specifications:
The component is a microchip or module with a red casing, and it has a few pins on the bottom. It is designed to be used in IoT applications, and its exact technic

In [3]:
### VLM Zero shot Water Level Detection Sensor Module

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: Water Level Detection Sensor Module. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"

image_file = "../../images_jpg/Water_Level_Detection_Sensor_Module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/Water_Level_Detection_Sensor_Module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7A7E820F7B20>]
image sizes:  [(4032, 3024)]
1. Description:
The Water Level Detection Sensor Module is an IoT component designed to measure the water level in a tank or container. It consists of a sensor that detects the water level and an electronic circuit that converts the sensor's output into a digital signal. The module can be easily integrated into various IoT applications, such as monitoring water levels in a swimming pool, a water tank, or a water reservoir.

2. Applications:
The Water Level Detection Sensor Module can be used in a variety of applications, including:
- Monitoring water levels in swimming pools, ensuring that the water level remains within a safe range for swimming and reducing the risk of accidents.
- Tracking water levels in water tanks, such as those in residential or commercial buildings, to prevent overflows and water dama

In [2]:
### VLM One shot Water Level Detection Sensor Module

model_path = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: Water Level Detection Sensor Module. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications.\n Here is an example for reference,\nULN 2003 Stepper Motor Driver Module:\nDescription:\nThe ULN2003 Stepper Motor Driver Module is an integrated circuit module designed to control bipolar stepper motors conveniently. It utilizes the ULN2003A Darlington array as its core component. This module provides an easy interface for controlling stepper motors, offering both simplicity and efficiency in motor control applications.\nApplications:\nThe ULN2003 Stepper Motor Driver Module finds widespread use in various projects and applications, including:\nRobotics: for precise control of robot movements.\nCNC machines: for controlling the motion of axes.\n3D printers: for controlling the position of the printing head.\nAutomated systems: for controlling mechanisms like conveyor belts and automated doors.\nDIY projects: for hobbyists and enthusiasts working on motion control projects.\nTechnical Specifications:\nOperating Voltage: 5V DC\nOutput Current: up to 500mA per channel (with proper heat sinking)\nDriver IC: ULN2003A Darlington Array\nNumber of Channels: 7 (suitable for driving bipolar stepper motors)\nCompatible Motors: Bipolar stepper motors with 4 or 6 wires\nControl Interface: 4-phase, 5-wire interface (can be easily interfaced with microcontrollers like Arduino)\nDimensions: Compact module size for easy integration into projects\nProtection Features: Built-in transient suppression diodes for protecting the module and connected components against voltage spikes\nMounting Holes: Provided for easy installation and integration into projects"
# prompt = "Can you make a short description about the part: HiLetgo GY-521 module.\n It should be divided into 3 sections as follows: \n1) Description \n2) Applications \n3) technical specification"
image_file = "../../images_jpg/Water_Level_Detection_Sensor_Module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": None,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 1000
})()

eval_model(args)

model_name:  llava-v1.5-7b


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


image files:  ['../../images_jpg/Water_Level_Detection_Sensor_Module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x782204C3FA00>]
image sizes:  [(4032, 3024)]
Water Level Detection Sensor Module:

Description:
The Water Level Detection Sensor Module is an electronic component designed to detect changes in water levels. It consists of a water-resistant sensor that measures the water level and an integrated circuit that converts the sensor's output into a digital signal. This module is commonly used in various applications, such as monitoring water levels in tanks, ponds, or aquariums.

Applications:
The Water Level Detection Sensor Module can be used in a wide range of applications, including:

1. Automated irrigation systems: To ensure that plants receive the appropriate amount of water and to prevent overwatering or underwatering.
2. Aquariums: To monitor water levels and prevent the fish from being exposed to too much or too little water.
3. Water storage tanks:

In [2]:
### VLM finetuning Water Level Detection Sensor Module

# model_path = "llava/checkpoints/llava-v1.5-7b-task-lora-first"
model_path = "llava/checkpoints/llava-v1.5-7b-lora"
model_basee = "liuhaotian/llava-v1.5-7b"
# prompt = "What are the things I should be cautious about when I visit here?"
# image_file = "https://llava-vl.github.io/static/images/view.jpg"
prompt = "You are an expert in IoT and hardware with extensive experience. You have been given an IoT kit component: Water Level Detection Sensor Module. Give a description of the component having 3 sections as follows: \n1) Description \n2) Applications \n3) Technical specifications"
image_file = "../../images_jpg/Water_Level_Detection_Sensor_Module.jpg"


args = type('Args', (), {
    "model_path": model_path,
    "model_base": model_basee,
    "model_name": get_model_name_from_path(model_path),
    "query": prompt,
    "conv_mode": None,
    "image_file": image_file,
    "sep": ",",
    "temperature": 0.1,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 512
})()

eval_model(args)

model_name:  llava-v1.5-7b-lora


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading LLaVA from base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...
image files:  ['../../images_jpg/Water_Level_Detection_Sensor_Module.jpg']
images:  [<PIL.Image.Image image mode=RGB size=4032x3024 at 0x7EA4B3047BE0>]
image sizes:  [(4032, 3024)]
Description:
The Water Level Detection Sensor Module is a compact and versatile electronic device designed to detect changes in water levels. It typically consists of a float arm, a conductive material, and a microcontroller or sensor chip. The float arm is connected to the conductive material, which is in turn connected to the microcontroller or sensor chip. When the water level rises, the conductive material comes into contact with the water, completing an electrical circuit and triggering an output signal.
Applications:
The Water Level Detection Sensor Module finds applications in various industries and scenarios where monitoring water levels is critical. Some common applications include:
Industrial Proc